In [149]:
import numpy as np
import pandas as pd
from datetime import date
import collections
import datetime
import os
import xarray as xr
import csv

## Generic the filepath to the main data folder
fpath0 = '/Users/finnwimberly/Library/CloudStorage/GoogleDrive-fwimberly@middlebury.edu/My Drive/'
fpath1 = 'Lizz Research Stuff/CSV Outputs/Initial Areas/Results_PET_PREC/'

modelnames = ['BCC.BCC-CSM2-MR', 'MPI-M.MPI-ESM1-2-HR', 'MRI.MRI-ESM2-0', 'NCAR.CESM2-WACCM']

scenarios = ['Ssp2p6', 'Ssp4p5', 'Ssp7p0', 'Ssp8p5']

basins = {'RHINE':'6242', 'RHONE':'6243','PO':'6241', 'DANUBE':'6202', 'TITICACA':'3912', 'SANTA':'3425', 
            'OCONA':'3418', 'MAJES':'3416', 'MAGDALENA':'3227', 'AMAZON':'3203', 'YELCHO':'3429', 
            'VALDIVIA':'3428', 'SERRANO':'3426', 'RAPEL':'3423', 'PUELO':'3422', 'PASCUA':'3420', 
            'PALENA':'3419', 'HUASCO':'3412', 'COPIAPO':'3409', 'CISNES':'3408', 'BIOBIO':'3405', 'BAKER':'3404',
            'AZOPARDO':'3403', 'AISEN':'3401', 'SANTA CRUZ':'3244', 'NEGRO':'3232', 'COLORADO':'3212', 
            'CHICO':'3209', 'TORNEALVEN':'6255', 'THJORSA':'6254', 'OLFUSA':'6237', 'LULEALVEN':'6227', 
            'KUBAN':'6223', 'KALIXALVEN':'6219', 'GLOMAA':'6213', 'DRAMSELVA':'6209', 'SVARTA':'6110', 
            'LAGARFLJOT':'6104', 'JOKULSA A FJOLLUM':'6101', 'CLUTHA':'5406', 'YUKON':'4435', 'TAKU':'4431', 
             'SUSITNA':'4430','STIKINE':'4428', 'SKEENA':'4427','SKAGIT':'4426','NUSHAGAK':'4418','NASS':'4416',
            'KUSKOKWIM':'4414','FRASER':'4410', 'COPPER':'4408', 'COLUMBIA':'4406', 'ALSEK':'4401', 'NELSON':'4125', 
              'MACKENZIE':'4123','COLVILLE':'4110', 'YSYK-KOL':'2919', 'UVS NUUR':'2918', 'TARIM HE':'2914', 
              'TALAS':'2913', 'LAKE BALKHASH':'2910','HAR US NUUR':'2909', 'CHUY':'2905', 'ARAL SEA':'2902', 
              'YELLOW RIVER':'2434', 'MEKONG':'2421', 'KAMCHATKA':'2413', 'SALWEEN':'2319', 'IRRAWADDY':'2310', 
              'INDUS':'2309', 'GANGES':'2306','BRAHMAPUTRA':'2302', 'OB':'2108', 'INDIGIRKA':'2103','YANGTZE' : '2433'}

In [150]:
#Loading in files
precip = {}
for m, model in enumerate(modelnames):
    precip[model[0:3]] = {}
    for s, SSP in enumerate(scenarios):
        precip[model[0:3]][SSP] = np.loadtxt(fpath0 + fpath1 + model + '_PREC_' + SSP + '.txt')

In [151]:
# Create new index using pandas date_range function
start_date = datetime.date(1900, 1, 1)
end_date = datetime.date(2100, 12, 1)
timesteps = pd.date_range(start_date, end_date, freq='MS').strftime('%Y-%m').tolist()

# Apply new index and datetime conversion
for m, model in enumerate(modelnames):
    for s, SSP in enumerate(scenarios):
        precip[model[0:3]][SSP] = pd.DataFrame(precip[model[0:3]][SSP])
        precip[model[0:3]][SSP].index = basins
        precip[model[0:3]][SSP].columns = timesteps

In [152]:
#Selecting only the 21st century and resampling to annual
start_2000 = pd.to_datetime("2000-01-01")
end_2100 = pd.to_datetime("2100-12-31")
precip_2000_2100 = {}
for m, model in enumerate(modelnames):
    precip_2000_2100[model[0:3]] = {}
    for s, SSP in enumerate(scenarios):
        columns_2000_2100 = [col for col in precip[model[0:3]][SSP].columns if start_2000 <= pd.to_datetime(col) <= end_2100]
        temp_df = pd.DataFrame(precip[model[0:3]][SSP][columns_2000_2100])
        temp_df.columns = pd.to_datetime(temp_df.columns)   #Not redundant... somehow datetime is lost?
        precip_2000_2100[model[0:3]][SSP] = temp_df.resample('Y', axis=1).sum()

In [167]:
from scipy.stats import linregress

years = np.arange(2000, 2101)

precip_trend = {}

for m, model in enumerate(modelnames):
    precip_trend[model[0:3]] = {}
    for s, SSP in enumerate(scenarios):
        precip_trend[model[0:3]][SSP] = {}
        for basin in basins:
            data = precip_2000_2100[model[0:3]][SSP].loc[basin]
            
            # Perform linear regression
            slope, intercept, r_value, p_value, std_err = linregress(years, data)

            precip_trend[model[0:3]][SSP][basin] = slope

In [200]:
avg_precip_trend = {}  # Create a dictionary to store the average trends for each basin

for basin in basins:
    sum_trends = 0  # Initialize the average trend for the current basin
    avg_trend = 0
    for m, model in enumerate(modelnames):
        for s, SSP in enumerate(scenarios):
            if basin in precip_trend[model[0:3]][SSP]:
                sum_trends += precip_trend[model[0:3]][SSP][basin]

        avg_trend = sum_trends / (len(scenarios) * len(modelnames)) # Calculate the average trend
        avg_precip_trend[basin] = avg_trend  # Store the average trend in the dictionary

In [207]:
#Combining our results into a single dataframe
out_df = pd.DataFrame.from_dict(avg_precip_trend, orient='index', columns=['Average Precipitation Trend'])

# Define the directory to save the CSV files
output_dir = '/Users/finnwimberly/Desktop/Lizz Research/CSV Outputs/Parameters/'

fname = f"Precip_Trend_Avg.csv"

# Define the full path of the output file
output_path = os.path.join(output_dir, fname)

# Save the DataFrame as CSV
out_df.to_csv(output_path, header=True, index=True)